# Инициализация

In [0]:
#@markdown - **Монтирование GoogleDrive** 
from google.colab import drive
drive.mount('GoogleDrive')

In [0]:
# #@markdown - **Размонтирование**
# !fusermount -u GoogleDrive

# Область кодов

In [0]:
#@title Метод опорных векторов { display-mode: "both" }
# Support Vector Machines
# В программе реализован метод опорных векторов с помощью градиентного спуска
#@markdown [Литература](https://en.wikipedia.org/wiki/Support-vector_machine)
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data

tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
#@markdown - **Таймер-декоратор**
import time
def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        t = end_time - start_time
        print('This operation takes {} s.'.format(t))
        return result
    return wrapper

In [0]:
#@markdown - **Настройка параметров**
batch_size = 128 #@param {type: "integer"}
num_epochs = 100 #@param {type: "integer"}
C_param = 0.1 #@param {type: "number"}
Reg_param = 1.0 #@param {type: "number"}
delta = 1.0 #@param {type: "number"}
learning_rate = 1e-3 #@param {type: "number"}


In [0]:
#@markdown - **Функция ошибки и функция точности**
def loss_fn(W,b,x_data,y_target):
    logits = tf.subtract(tf.matmul(x_data, W),b)
    norm_term = tf.divide(tf.reduce_sum(tf.multiply(tf.transpose(W),W)),2)
    classification_loss = tf.reduce_mean(tf.maximum(0., tf.subtract(delta, tf.multiply(logits, y_target))))
    total_loss = tf.add(tf.multiply(C_param,classification_loss), tf.multiply(Reg_param,norm_term))
    return total_loss

def inference_fn(W,b,x_data,y_target):
    prediction = tf.sign(tf.subtract(tf.matmul(x_data, W), b))
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, y_target), tf.float32))
    return accuracy

# Извлечение индексов изображений 0 или 1
@timer
def extraction_fn(data):
    index_list = []
    for idx in range(data.shape[0]):
        if data[idx] == 0 or data[idx] == 1:
            index_list.append(idx)
    return index_list

In [0]:
#@markdown - **Извлечение обучающих и тестовых изображений**
mnist = input_data.read_data_sets("sample_data/MNIST_data", reshape=True, one_hot=False)
data = {}
data['train_image'] = mnist.train.images
data['train_label'] = mnist.train.labels
data['test_image'] = mnist.test.images
data['test_label'] = mnist.test.labels

index_list_train = extraction_fn(data['train_label'])
index_list_test = extraction_fn(data['test_label'])

data['train_image'] = mnist.train.images[index_list_train]
data['train_label'] = mnist.train.labels[index_list_train]
data['test_image'] = mnist.test.images[index_list_test]
data['test_label'] = np.array(mnist.test.labels[index_list_test], dtype=np.float32)
# data['test_label'] = mnist.test.labels[index_list_test].astype('float32')

data['train_image_label'] = np.c_[data['train_image'], data['train_label']]
num_samples, num_features = data['train_image'].shape
num_hidden = 256


Extracting sample_data/MNIST_data/train-images-idx3-ubyte.gz
Extracting sample_data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting sample_data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting sample_data/MNIST_data/t10k-labels-idx1-ubyte.gz
This operation takes 0.1576387882232666 s.
This operation takes 0.02683234214782715 s.


In [0]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('Input'):
        x_data = tf.placeholder(shape=[None, num_features], dtype=tf.float32)
        y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)
    with tf.name_scope('Net'):
        # Снижение размерности входных данных
        w_1 = tf.Variable(tf.random_normal(shape=[num_features, num_hidden]), name='w_1')
        b_1 = tf.Variable(tf.random_normal(shape=[num_hidden]), name='b_1')
        layer_1 = tf.sigmoid(tf.matmul(x_data, w_1) + b_1)

        W = tf.Variable(tf.random_normal(shape=[num_hidden, 1]), name='weights')
        b = tf.Variable(tf.random_normal(shape=[1]), name='bias')
    with tf.name_scope('Loss'):
        total_loss = loss_fn(W, b, layer_1, y_target)
    with tf.name_scope('Accuracy'):
        prediction = tf.sign(tf.subtract(tf.matmul(layer_1, W), b))
        accuracy = inference_fn(W, b, layer_1, y_target)
    with tf.name_scope('Train'):
        train_op = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

In [0]:
#@markdown - **Обучение сетей**
test_label = data['test_label'].reshape(-1, 1)
test_label[test_label==0] = -1
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(num_epochs):
        np.random.shuffle(data['train_image_label'])
        image_batch = data['train_image_label'][:batch_size,:-1]
        label_batch = data['train_image_label'][:batch_size,-1]
        label_batch[label_batch==0] = -1

        _, loss, acc = sess.run([train_op, total_loss, accuracy], feed_dict={x_data: image_batch, 
                                                                            y_target: label_batch.reshape(-1, 1)})
        acc *= 100
        if (epoch + 1) % 5 == 0:
            test_loss, test_acc = sess.run([total_loss, accuracy], feed_dict={x_data: data['test_image'], 
                                                                            y_target: test_label})
            test_acc *= 100
            print_list = [epoch + 1, loss, acc, test_acc]
            print('Epoch {0[0]}, loss: {0[1]:.4f}, training accuracy: {0[2]:.2f}%.'.format(print_list))
            print(' '*10, 'Testing accuracy is {0[3]:.2f}%.'.format(print_list))
sess.close()   

Epoch 5, loss: 27.1712, training accuracy: 50.00%.
           Testing accuracy is 48.89%.
Epoch 10, loss: 18.6308, training accuracy: 64.84%.
           Testing accuracy is 66.15%.
Epoch 15, loss: 11.8959, training accuracy: 81.25%.
           Testing accuracy is 80.43%.
Epoch 20, loss: 6.9757, training accuracy: 89.06%.
           Testing accuracy is 89.36%.
Epoch 25, loss: 3.6435, training accuracy: 92.19%.
           Testing accuracy is 94.80%.
Epoch 30, loss: 1.6239, training accuracy: 95.31%.
           Testing accuracy is 96.55%.
Epoch 35, loss: 0.5562, training accuracy: 96.88%.
           Testing accuracy is 97.16%.
Epoch 40, loss: 0.1190, training accuracy: 97.66%.
           Testing accuracy is 97.83%.
Epoch 45, loss: 0.0032, training accuracy: 99.22%.
           Testing accuracy is 98.16%.
Epoch 50, loss: 0.0276, training accuracy: 99.22%.
           Testing accuracy is 98.35%.
Epoch 55, loss: 0.0705, training accuracy: 97.66%.
           Testing accuracy is 98.44%.
Epoch 60